# 02 — Model Training / Upsert\nCompute text embeddings and build local FAISS index.

In [ ]:
import pandas as pd, faiss, json
from sentence_transformers import SentenceTransformer
df = pd.read_csv('../data/raw/intern_data_ikarus.csv')
df['price_num'] = pd.to_numeric(df['price'].astype(str).str.replace(r'[^\d.]','',regex=True), errors='coerce')
df['text_blob'] = (df['title'].fillna('') + '. ' + df['brand'].fillna('') + '. ' + df['material'].fillna('') + '. ' + df['description'].fillna('')).str.strip()
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
emb = model.encode(df['text_blob'].fillna('').tolist(), convert_to_numpy=True, normalize_embeddings=True)
index = faiss.IndexFlatIP(emb.shape[1])
index.add(emb.astype('float32'))
faiss.write_index(index, '../backend/app/local.faiss')
md = df[['uniq_id','title','brand','material','price_num','categories','images','color']].fillna('').to_dict(orient='records')
open('../backend/app/local_meta.json','w').write(json.dumps(md))
emb.shape
